In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from itertools import chain
from torchdiffeq import odeint
from lib.models import Fp, Fa, FaFp, Encoder_z0_RNN
import tqdm
import lib.utils as utils
from lib import train_functions as train_functions
from lib.osthus_stuff import make_ics, reparam
from torch.distributions import Normal, kl_divergence
device = 'cpu'
import torch 
torch.set_num_threads(1)
from torch import nn

In [ ]:
import lib.Data_Constructor as Data_Constructor
import lib.utils as utils
import copy
import datetime as dt
import pandas as pd
import numpy as np
import tqdm

import torch
import torch.nn as nn
from torch.distributions import Categorical, Normal, kl_divergence
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from itertools import chain
import lib.utils as utils
import lib.train_functions as train_functions
from torchdiffeq import odeint
import tqdm
import lib.models as models
torch.set_num_threads(1)
import os

In [15]:
n_samples = 128
latent_dim = 8
hidden_dim = 64
epochs = 1000
lr=1e-3
lr_scale = 0.999
norm_weighting = 1e-1
Fa_scale = 0.2
disable = True

ls = []
for n_samples in [128]:
    for latent_dim in [4,8,16]:
        for hidden_dim in [16,32,64,128]:
            for epochs in [2500, 6000]:
                for lr in [1e-3, 5e-4, 1e-4]:
                    for lr_scale in [0.999, 0.9999, 1.0]:
                        for norm_weighting in [1.0, 1e-1, 1e-2, 1e-3]:
                            for Fa_scale in [0.2]:
                                ls.append({'n_samples':n_samples,'latent_dim':latent_dim, 'hidden_dim':hidden_dim, 'epochs':epochs,'lr':lr,'lr_scale':lr_scale, 'norm_weighting':norm_weighting, 'Fa_scale':Fa_scale})  


df = pd.DataFrame.from_dict(ls)
df['started'] = np.zeros(df.shape[0])
df['validation_score'] = np.zeros(df.shape[0])

df.index = np.linspace(0, df.shape[0]-1, df.shape[0]).astype(int)
df.to_csv('validation_scores.csv')                    

In [ ]:
# Parameters for the model
input_size = 11  # Number of features (time series)
hidden_size = 32  # Number of features in the hidden state
num_layers = 2  # Number of stacked LSTM layers
latent_dim = 8  # Number of output classes
n_samples = 64
window_size = 28
batch_size = 32
root = 'checkpoints/SIR_Qs/'
epochs = params['epochs']







In [ ]:
def evaluate(params, disable=True, quick=False):
    input_size = int(params['input_size'])
    hidden_size = int(params['hidden_size'])
    num_layers = int(params['num_layers'])
    latent_dim = int(params['latent_dim'])
    n_samples = params['n_samples']
    window_size = params['window_size']
    batch_size = params['batch_size']
    root = params['root']

    
    test_season = 2016
    data_season = 2015
    means=[0.8, 0.55]
    stds = [0.2, 0.2]
    lr = 1e-3
    device='cpu'
    gamma = 56
    tmax = 56
       


    _data = Data_Constructor.DataConstructor(test_season, data_season, gamma, window_size, n_queries=input_size-1, selection_method='distance')
    _data()
    inputs, outputs, dates, test_dates = _data.build()
    
    test_start = np.where(dates == test_dates[0])[0][0]
    dtype = torch.float32
    x_tr = torch.tensor(inputs[:test_start], dtype=dtype)
    x_test = torch.tensor(inputs[test_start:], dtype=dtype)
    y_tr = torch.tensor(outputs[:test_start], dtype=dtype)
    y_test = torch.tensor(outputs[test_start:], dtype=dtype)
    
    n_batches = int(np.ceil(x_tr.shape[0]/batch_size))
    # batch it all 
    x_train = []
    y_train = []
    for b in range(n_batches):
        x_train.append(torch.tensor(x_tr[b*batch_size:(b+1)*batch_size], dtype=dtype))
        y_train.append(torch.tensor(y_tr[b*batch_size:(b+1)*batch_size], dtype=dtype).unsqueeze(-2))
    
    # Create the model
    n_regions = 1
    enc = Encoder_BiDirectionalLSTM(input_size, hidden_size, num_layers, latent_dim-1)
    ode = Fp(n_regions, latent_dim, nhidden=32)
    dec = models.Decoder(n_regions, 3, 1)
    
    optimizer = torch.optim.Adam(enc.parameters(), lr=1e-3)
    pbar = tqdm.trange(30)
    for epoch in pbar:
        kls = 0
        for x_tr in x_train:
            optimizer.zero_grad()
            
            mean, std = enc(x_tr)
            prior = make_prior(mean, latent_dim=latent_dim)
            kl = kl_divergence(Normal(mean, std), prior).mean(0).sum()
            kl.backward()
            optimizer.step()
            kls += kl.detach().numpy()
            
        pbar.set_postfix({'KL_z':kls})
    
    optimizer = torch.optim.Adam(chain(enc.parameters(), ode.parameters(), dec.parameters()), lr=1e-3)
    _history = train_functions.history()
    
        
    
    epochs = 2000
    pbar = tqdm.trange(epochs)
    for epoch in pbar:
        
        t = torch.linspace(1, tmax, tmax)/7
        epc = len(_history.epoch_history)
        for x_tr, y_tr in zip(x_train, y_train):
            batch_size = x_tr.shape[0]
            eps = torch.tensor(np.random.normal(0, 1,  (n_samples, batch_size, n_regions, latent_dim-1)), dtype=dtype, device=device)
            ode.clear_tracking()
            
            optimizer.zero_grad()
    
            
            mean, std = enc(x_tr)
            z = reparam(eps, std, mean, n_samples, batch_size)
            latent = odeint(ode, z, t, method='rk4', options=dict(step_size = 1.0))
            y_pred = dec(latent[..., :3]).reshape((tmax, n_samples, batch_size, n_regions)).permute(2,1,0,3)
            
            nll = train_functions.nll_loss(y_pred, y_tr[:, :t.shape[0], :])
            kl_p = train_functions.get_kl_params(1, ode.posterior(), means=means, stds = stds,limit = 500)
            kl_z = kl_divergence(make_prior(mean, latent_dim=latent_dim), Normal(mean, std)).sum(-1).mean() / len(x_train)
            reg_loss = train_functions.latent_init_loss(latent[..., :3])
    
            loss = nll+kl_p+kl_z+reg_loss
            loss.backward()
            optimizer.step()
            _history.batch([loss.cpu(), nll.cpu(), kl_z.cpu(),kl_p.cpu(),reg_loss.cpu(), optimizer.param_groups[-1]['lr'], tmax], ['loss', 'nll', 'kl_latent', 'kl_params', 'reg_loss', 'lr', 'tmax'])
        _history.reset()
        if epoch > 10:
            if np.all([h['nll'] < -2 for h in _history.epoch_history[-10:]]):
                tmax = min(tmax+1, 28)
        
        utils.update_learning_rate(optimizer, 0.999, lr/10)
        
        if not os.path.exists(root):
            os.mkdir(root)
        torch.save(enc.state_dict(), root+'enc_' + '.pth')
        torch.save(ode.state_dict(), root+'sir_' + '.pth')
        torch.save(dec.state_dict(), root+'dec_' + '.pth')
    
        pbar.set_postfix(_history.epoch_history[-1])


    mean, std = enc(torch.tensor(x_test))
    batch_size = x_test.shape[0]
    eps = torch.tensor(np.random.normal(0, 1,  (n_samples, batch_size, n_regions, latent_dim-1)), dtype=dtype, device=device)
    z = reparam(eps, std, mean, n_samples, batch_size)
    latent = odeint(ode, z, t, method='rk4', options=dict(step_size = 1.0))
    y_pred = dec(latent[..., :3]).reshape((tmax, n_samples, batch_size, n_regions)).permute(2,1,0,3)*7.7151
    
    score = train_functions.nll_loss(y_pred, y_test.unsqueeze(-2)).detach().numpy()

    return score

In [12]:

def evaluate(params, disable=True, quick=False):
    ili = pd.read_csv('Data/national_flu.csv', index_col = -1, parse_dates=True)

    starts = ili[ili['week'] == 40].index
    ends = [i + dt.timedelta(weeks=34) for i in starts]

    starts = starts[[1,2,3,4,5,6,7,8,9,10,13,14,15,16,17,18]]
    ends = np.asarray(ends)[[1,2,3,4,5,6,7,8,9,10,13,14,15,16,17,18]]

    x_train = []
    for start, end in zip(starts, ends):
        x_train.append(torch.tensor(ili.loc[start:end, 'weighted_ili'].values/100, dtype=torch.float32))
    
    n_samples = int(params['n_samples'])
    latent_dim = int(params['latent_dim'])
    hidden_dim = int(params['hidden_dim'])
    epochs = int(params['epochs'])
    lr = params['lr']
    lr_scale = params['lr_scale']
    norm_weighting = params['norm_weighting']
    Fa_scale = params['Fa_scale']
    
    if quick:
        epochs=1
    
    dtype = torch.float32
    t = torch.linspace(1,35,35, dtype=dtype)

    ode = FaFp(latent_dim, hidden_dim, Fa_scale = Fa_scale)
    enc = Encoder_z0_RNN(latent_dim, 1, Fp =True)

    _history = train_functions.history()
    optimizer = torch.optim.RAdam(list(enc.parameters()) + list(ode.parameters()), lr=lr)

    ode.Fa_scale = Fa_scale


    pbar = tqdm.trange(150, disable=disable)
    for epoch in pbar:
        for x in x_train[:-4]:        
            IC_p = make_ics(x[0], I_0 = 0.88, std = 0.002, n = n_samples, latent=3)

            optimizer.zero_grad()
            mean, std = enc(x.unsqueeze(0).unsqueeze(-1))

            loss = 1e-4*kl_divergence(Normal(mean.squeeze()[..., :2], std.squeeze()[..., :2]), 
                                 Normal(IC_p.mean(0)[..., :2], IC_p.std(0)[..., :2])).sum()

            loss.backward()
            optimizer.step()

            batch_data = {'kl_z':loss.cpu()}
            _history.batch(batch=batch_data)
            utils.update_learning_rate(optimizer, 1.00, optimizer.param_groups[-1]['lr']/10)
        _history.reset()
        pbar.set_postfix(_history.epoch_history[-1])

    pbar = tqdm.trange(epochs, disable=disable)
    t = torch.linspace(1,35,35, dtype=dtype)
    for epoch in pbar:
        for x in x_train[:-4]:        
            IC_p = make_ics(x[0], I_0 = 0.88, std = 0.002, n = n_samples, latent=3)
            z = torch.tensor(np.random.normal(0, 1, (n_samples, latent_dim-1)), dtype=dtype, device=device)

            ode.clear_tracking()
            optimizer.zero_grad()
            mean, std = enc(x[:5].unsqueeze(0).unsqueeze(-1))
            IC = reparam(mean, std, z)

            latent = odeint(ode, IC, t, method='rk4', options=dict(step_size = 1.0))

            y_mean = latent.mean(1)[...,1]
            y_std = latent.std(1)[...,1]

            nll = torch.tensor(0)
            if len(_history.epoch_history) > 200:
                nll = torch.mean(-Normal(y_mean, y_std).log_prob(x))

            kl_p = kl_divergence(Normal(torch.stack(ode.params).mean(1), torch.stack(ode.params).std(1, unbiased=False)), 
                                 Normal(torch.tensor([0.8, 0.55]), torch.tensor([0.25, 0.25]))).mean()
            kl_z = 1e-1*kl_divergence(Normal(mean.squeeze()[..., :2], std.squeeze()[..., :2]), 
                                 Normal(IC_p.mean(0)[..., :2], IC_p.std(0)[..., :2])).sum()
            norm_loss = norm_weighting*torch.sqrt(torch.sum(torch.square(torch.stack(ode.Fa_track)[..., :2])))

            loss = 10*nll + kl_p + kl_z + norm_loss
            loss.backward()
            optimizer.step()

            batch_data = {'Loss':loss.cpu(), 'nll':nll.cpu(), 'kl_p':kl_p.cpu(), 'kl_z':kl_z.cpu(),'norm_loss':norm_loss.cpu(), 'lr':optimizer.param_groups[-1]['lr']}
            _history.batch(batch=batch_data)
            utils.update_learning_rate(optimizer, lr_scale, optimizer.param_groups[-1]['lr']/10)
        _history.reset()
        pbar.set_postfix(_history.epoch_history[-1])

    score = 0
    for x in x_train[-4:]:        
        z = torch.tensor(np.random.normal(0, 1, (n_samples, latent_dim-1)), dtype=dtype, device=device)

        ode.clear_tracking()
        mean, std = enc(x[:5].unsqueeze(0).unsqueeze(-1))
        IC = reparam(mean, std, z)

        latent = odeint(ode, IC, t, method='rk4', options=dict(step_size = 1.0))

        y_mean = latent.mean(1)[...,1]
        y_std = latent.std(1)[...,1]

        nll = torch.mean(-Normal(y_mean, y_std).log_prob(x))
        score += float(nll.detach())
        
    return score



In [13]:
import time
import random
from filelock import FileLock
import sys
import time
import argparse

position = 1


lock = FileLock("validation_scores.csv.lock")

with lock:
    df = pd.read_csv('validation_scores.csv', index_col=0)
    try:
        param_num = np.min(np.where(df['started'] == 0))
    except:
        print('oh no')
    df.loc[param_num, 'started'] = 1
    df.to_csv('validation_scores.csv')

score = 10
try:
    score = evaluate(dict(df.loc[param_num]), disable=False)
except Exception as e:
    print("An error occurred:", e)


with lock:
    df = pd.read_csv('validation_scores.csv', index_col=0)
    df.loc[param_num, 'score'] = score
    df.to_csv('validation_scores.csv')

100%|██████████| 55/55 [01:18<00:00,  1.43s/it, Loss=362, nll=3.61, kl_p=2.06, kl_z=324, norm_loss=0.0129, lr=0.00052]  


13.213887482881546


In [6]:
score

10

In [ ]:
def evaluate(params, disable=disable):
    ili = pd.read_csv('Data/national_flu.csv', index_col = -1, parse_dates=True)

    starts = ili[ili['week'] == 40].index
    ends = [i + dt.timedelta(weeks=34) for i in starts]

    starts = starts[[1,2,3,4,5,6,7,8,9,10,13,14,15,16,17,18]]
    ends = np.asarray(ends)[[1,2,3,4,5,6,7,8,9,10,13,14,15,16,17,18]]

    x_train = []
    for start, end in zip(starts, ends):
        x_train.append(torch.tensor(ili.loc[start:end, 'weighted_ili'].values/100, dtype=torch.float32))
    
    n_samples = params['n_samples']
    latent_dim = params['latent_dim']
    hidden_dim = params['hidden_dim']
    epochs = params['epochs']
    lr = params['lr']
    n_samples = params['n_samples']
    lr_scale = params['lr_scale']
    norm_weighting = params['norm_weighting']
    Fa_scale = params['Fa_scale']
    
    dtype = torch.float32
    t = torch.linspace(1,35,35, dtype=dtype)

    ode = FaFp(latent_dim, hidden_dim, Fa_scale = Fa_scale)
    enc = Encoder_z0_RNN(latent_dim, 1, Fp =True)

    _history = train_functions.history()
    optimizer = torch.optim.RAdam(list(enc.parameters()) + list(ode.parameters()), lr=lr)

    ode.Fa_scale = Fa_scale

    pbar = tqdm.trange(150, disable=disable)
    for epoch in pbar:
        for x in x_train[:-4]:        
            IC_p = make_ics(x[0], I_0 = 0.88, std = 0.002, n = n_samples, latent=3)

            optimizer.zero_grad()
            mean, std = enc(x.unsqueeze(0).unsqueeze(-1))

            loss = 1e-4*kl_divergence(Normal(mean.squeeze()[..., :2], std.squeeze()[..., :2]), 
                                 Normal(IC_p.mean(0)[..., :2], IC_p.std(0)[..., :2])).sum()

            loss.backward()
            optimizer.step()

            batch_data = {'kl_z':loss.cpu()}
            _history.batch(batch=batch_data)
            utils.update_learning_rate(optimizer, 1.00, optimizer.param_groups[-1]['lr']/10)
        _history.reset()
        pbar.set_postfix(_history.epoch_history[-1])

    pbar = tqdm.trange(epochs, disable=disable)
    t = torch.linspace(1,35,35, dtype=dtype)
    for epoch in pbar:
        for x in x_train[:-4]:        
            IC_p = make_ics(x[0], I_0 = 0.88, std = 0.002, n = n_samples, latent=3)
            z = torch.tensor(np.random.normal(0, 1, (n_samples, 7)), dtype=dtype, device=device)

            ode.clear_tracking()
            optimizer.zero_grad()
            mean, std = enc(x[:5].unsqueeze(0).unsqueeze(-1))
            IC = reparam(mean, std, z)

            latent = odeint(ode, IC, t, method='rk4', options=dict(step_size = 1.0))

            y_mean = latent.mean(1)[...,1]
            y_std = latent.std(1)[...,1]

            nll = torch.tensor(0)
            if len(_history.epoch_history) > 200:
                nll = torch.mean(-Normal(y_mean, y_std).log_prob(x))

            kl_p = kl_divergence(Normal(torch.stack(ode.params).mean(1), torch.stack(ode.params).std(1, unbiased=False)), 
                                 Normal(torch.tensor([0.8, 0.55]), torch.tensor([0.25, 0.25]))).mean()
            kl_z = 1e-1*kl_divergence(Normal(mean.squeeze()[..., :2], std.squeeze()[..., :2]), 
                                 Normal(IC_p.mean(0)[..., :2], IC_p.std(0)[..., :2])).sum()
            norm_loss = norm_weighting*torch.sqrt(torch.sum(torch.square(torch.stack(ode.Fa_track)[..., :2])))

            loss = 10*nll + kl_p + kl_z + norm_loss
            loss.backward()
            optimizer.step()

            batch_data = {'Loss':loss.cpu(), 'nll':nll.cpu(), 'kl_p':kl_p.cpu(), 'kl_z':kl_z.cpu(),'norm_loss':norm_loss.cpu(), 'lr':optimizer.param_groups[-1]['lr']}
            _history.batch(batch=batch_data)
            utils.update_learning_rate(optimizer, lr_scale, optimizer.param_groups[-1]['lr']/10)
        _history.reset()
        pbar.set_postfix(_history.epoch_history[-1])

    score = 0
    for x in x_train[-4:]:        
        z = torch.tensor(np.random.normal(0, 1, (n_samples, 7)), dtype=dtype, device=device)

        ode.clear_tracking()
        mean, std = enc(x[:5].unsqueeze(0).unsqueeze(-1))
        IC = reparam(mean, std, z)

        latent = odeint(ode, IC, t, method='rk4', options=dict(step_size = 1.0))

        y_mean = latent.mean(1)[...,1]
        y_std = latent.std(1)[...,1]

        nll = torch.mean(-Normal(y_mean, y_std).log_prob(x))
        score += float(nll.detach())
        
    return score

In [ ]:
print(score)

In [ ]:
plt.plot((y_mean+y_std).detach())
plt.plot((y_mean-y_std).detach())
plt.plot((y_mean).detach())
plt.plot(x, color='red')
print(float(torch.mean(-Normal(y_mean, y_std).log_prob(x)).detach()))